In [15]:
import sys
sys.path.append("..")
import pandas as pd
import json
import os
from collections import defaultdict, Counter
from pprint import pprint

from src.LoraCreator import LORAFactory

from peft import PeftModel
from IPython.display import display

from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM, AutoTokenizer

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [2]:
foundation_model_path = "/home/ec2-user/Llama-2-13B-Chat-fp16"
finetuned_model_path = "/home/ec2-user/merged_llama2_by_files"
adapter_model_path = "/home/ec2-user/nghi/llama2_lora_test"

In [3]:
lora_factory = LORAFactory(foundation_model_path, finetuned_model_path, adapter_model_path, rank=2)

In [4]:
lora_factory.load_models()

loading foundational model ...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

loading fine-tuned model ...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

initialize adapter ...


In [5]:
lora_factory.create_adapter()

computing low rank matrix for model.layers.0.self_attn.q_proj ...
computing low rank matrix for model.layers.0.self_attn.v_proj ...
computing low rank matrix for model.layers.1.self_attn.q_proj ...
computing low rank matrix for model.layers.1.self_attn.v_proj ...
computing low rank matrix for model.layers.2.self_attn.q_proj ...
computing low rank matrix for model.layers.2.self_attn.v_proj ...
computing low rank matrix for model.layers.3.self_attn.q_proj ...
computing low rank matrix for model.layers.3.self_attn.v_proj ...
computing low rank matrix for model.layers.4.self_attn.q_proj ...
computing low rank matrix for model.layers.4.self_attn.v_proj ...
computing low rank matrix for model.layers.5.self_attn.q_proj ...
computing low rank matrix for model.layers.5.self_attn.v_proj ...
computing low rank matrix for model.layers.6.self_attn.q_proj ...
computing low rank matrix for model.layers.6.self_attn.v_proj ...
computing low rank matrix for model.layers.7.self_attn.q_proj ...
computing 

In [12]:
merged_lora_model = PeftModel.from_pretrained(
        lora_factory.foundation_model,
        lora_factory.adapter_model_path,
        # torch_dtype=torch.float16
    )

In [13]:
merged_lora_model_path = "/home/ec2-user/nghi/merged_llama2_by_files_LORA"
print("Applying the LoRA")
new_merged_model = merged_lora_model.merge_and_unload()

Applying the LoRA


In [16]:
base_tokenizer = AutoTokenizer.from_pretrained(lora_factory.foundation_model_path, use_fast=False, legacy=False)

In [17]:
new_merged_model.save_pretrained(merged_lora_model_path)
base_tokenizer.save_pretrained(merged_lora_model_path)

('/home/ec2-user/nghi/merged_llama2_by_files_LORA/tokenizer_config.json',
 '/home/ec2-user/nghi/merged_llama2_by_files_LORA/special_tokens_map.json',
 '/home/ec2-user/nghi/merged_llama2_by_files_LORA/tokenizer.model',
 '/home/ec2-user/nghi/merged_llama2_by_files_LORA/added_tokens.json')